# Оптимизация маршрута посещения точек интереса при помощи openrouteservice.org и ortools (загруженных из файла)


На примере построения маршрута по кафе в одном из районов Москвы покажем силу используемого инструмента. Вместо использования предоженных в сервисе мест, мы будем выбирать свои и загружать их из csv файла.

In [1]:
import folium
import pandas as pd
from shapely import wkt, geometry

При выборе района, в котором будем строить маршрут, обычно координаты пограничных точек записываются в строчку wkt_str, но я и их загружаю из csv файла (polygon.csv представляет себя один столбик из координат точек, которые стоят в углах выбранной зоны), почему бы и нет.

In [2]:
api_key = '5b3ce3597851110001cf62482441a14af0e1443fb908db2194c3dd22'
df = pd.read_csv('polygon.csv', sep=';')
df = df.values.tolist()
wkt_str = 'Polygon (('
for koord in df:
        wkt_str += koord[0].replace(',', '') + ', '
wkt_str = wkt_str[:-2]
wkt_str += '))'

aoi_geom = wkt.loads(wkt_str)  
aoi_coords = list(aoi_geom.exterior.coords)  
aoi_coords = [(y, x) for x, y in aoi_coords]  
aoi_centroid = aoi_geom.centroid  

Покажем выбранный район на карте

In [3]:
m = folium.Map(tiles='Stamen Toner', location=(aoi_centroid.y, aoi_centroid.x), zoom_start=14)
folium.vector_layers.Polygon(aoi_coords,
                             color='#ffd699',
                             fill_color='#ffd699',
                             fill_opacity=0.2,
                             weight=3).add_to(m)
m

Далее загружаем наши выбранные точки для посещения из csv файла(файл mesta.csv представляет из себя два столбика: место-координаты) и обрабатываем данные

In [4]:
from openrouteservice import client, places

ors = client.Client(key=api_key)

coffee_addresses = pd.read_csv('coffee.csv', sep=';')
koords = coffee_addresses['koordinaty'].values.tolist()
names = coffee_addresses['mesto'].values.tolist()
cof_coords = []

for koord in koords:
    nlist = []
    nlist = list(map(float, koord.replace(',', '').split()))
    cof_coords.append(nlist)

poi_addresses = []
for i in range(len(cof_coords)):
    lon = cof_coords[i][0]
    lat = cof_coords[i][1]
    name = names[i]
    popup = "<strong>{0}</strong><br>Lat : {1:.3f}<br>Long: {2:.3f}".format(name, lat, lon)
    icon = folium.map.Icon(color = 'lightgray',
    icon_color = '#b5231a',
    icon = 'coffee',
    prefix = 'fa')
    folium.map.Marker([lat , lon] , icon = icon , popup = popup ).add_to(m)
    poi_addresses.append(name)
m

Построим оптимальный маршрут и посчитаем время в пути (да, и кстати, в нашей модели мы не тратимся на такси, а гуляем по Москве) - мы заканчиваем путь в той точке, из которой стартовали.

In [5]:
request = {'locations': cof_coords,
    'profile': 'foot-walking',
    'metrics': ['duration']}
poi_matrix = ors.distance_matrix(**request)

from ortools.constraint_solver import pywrapcp
tsp_size = len(poi_addresses)
num_routes = 1
start = 0

optimal_coords = []

if tsp_size > 0:
    manager = pywrapcp.RoutingIndexManager(tsp_size, num_routes, start)
    routing = pywrapcp.RoutingModel(manager)

    
def distance_callback (from_index, to_index) :
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return int(poi_matrix['durations'][from_node][to_node])


transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
assignment = routing.Solve()
if assignment :
    print ("Длительность:" + str(round(assignment.ObjectiveValue(), 3) // 60) + "минуты\n")
    index = routing.Start(start)
    route = ''
    for node in range(routing.nodes()):
        optimal_coords.append(cof_coords[manager.IndexToNode(index)])
        route += str(poi_addresses[manager.IndexToNode(index)]) + '->'
        index = assignment.Value(routing.NextVar(index))
route += str(poi_addresses[manager.IndexToNode(index)])
optimal_coords.append(cof_coords[manager.IndexToNode(index)])
print ("Путь:\n" + route)

Длительность:84минуты

Путь:
Даблби->I Love cake->Альпака и кофе->CostaCofeee->Кофейня на Остоженке->Академия->Old Cringer->Шарлин->One Price Coffee->Шоколадница->Даблби


Теперь визиализируем случаный маршрут:

In [6]:
def style_function(color):
    return lambda feature: dict(color=color,
                                weight=3,
                                opacity=1)

cof_coords.append(cof_coords[0])
request = {'coordinates': cof_coords,
           'profile': 'foot-walking',
           'geometry': 'true',
           'format_out': 'geojson',
           #            'instructions': 'false'
           }
random_route = ors.directions(**request)

folium.features.GeoJson(data=random_route,
                        name='Random route',
                        style_function=style_function('#84e184'),
                        overlay=True).add_to(m)

m

Теперь наложим на него вычисленный оптимальный маршрут:

In [7]:
request['coordinates'] = optimal_coords
optimal_route = ors.directions(**request)
folium.features.GeoJson(data=optimal_route,
                        name='Optimal route',
                        style_function=style_function('#6666ff'),
                        overlay=True).add_to(m)

m.add_child(folium.map.LayerControl())
m

Видно, что оптимальный маршрут (синий) короче случайного (зеленого)

In [8]:
optimal_duration = 0
random_duration = 0

optimal_duration = optimal_route['features'][0]['properties']['summary']['duration'] // 60
random_duration = random_route ['features'][0]['properties']['summary']['duration'] // 60

print ("Длительность оптимального маршрута: {0:.0f} минут\nДлительность случайного маршрута: {1:.0f} минут".format(optimal_duration, random_duration))

Длительность оптимального маршрута: 84 минут
Длительность случайного маршрута: 142 минут


Получается, что мы сэкономили почти час! Но гулять полезно для здоровья. Поэтому, после прочтения - сжечь.